In [7]:
import numpy as np
import tensorflow as tf
from matplotlib import pyplot as plt
import sklearn as sk
import os
from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from numpy.random import uniform
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import unidecode
import pandas as pd
import nltk
import re
from os import listdir
from os.path import isfile, join
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
from sklearn.metrics import f1_score
from sklearn import svm
import pickle


https://drive.google.com/drive/folders/1qD0tbXwpeD-cU8phT_w9-flJoj_0a-0s?usp=sharing

<h3>
GO TO THIS LINK, DOWNLOAD THE 4 CSV FILES AND UPLOAD THEM TO THE DIRECTORY SEEN ON THE LEFT SIDE
CLICK THE ICON NAMED'FILES' AND CLICK UPLOAD ICON
</h3>

In [10]:
"""definitions of classifier and other functions, 
each call to the classifier function auto prints various metrics"""

def read_csv(file_name, sep=','):
    df = pd.read_csv(file_name, sep=sep)
    return df

def print_f1(Y_pred, Y_test):
    print (f1_score(Y_test, Y_pred))

def evaluate_model(Y_test,Y_pred):
    TP = ((Y_pred == Y_test) & (Y_pred > 0)).sum() #true positives
    TN = ((Y_pred == Y_test) & (Y_pred < 1)).sum() #true negatives
    FP = ((Y_pred != Y_test) & (Y_pred > 0)).sum() #false positives
    FN = ((Y_pred != Y_test) & (Y_pred < 1)).sum() #false negatives
    
    print("TP: ", TP)
    print("TN: ", TN)
    print("FP: ", FP)
    print("FN: ", FN)
    
    f1 = sk.metrics.f1_score(Y_test, Y_pred)
    print("f1 score is: ", f1)
    
def gnb_simple(X_train, Y_train, X_test, Y_test):
    clf = GaussianNB() #complement naive bayes classifier with different smoothing parameters

    Y_pred = clf.fit(X_train, Y_train).predict(X_test) #fitting and predicting
    f1 = sk.metrics.f1_score(Y_test, Y_pred)
    print("f1 score is: ", f1)
    
    evaluate_model(Y_test,Y_pred)
    return clf

def mnb_simple(X_train, Y_train, X_test, Y_test):
    clf = MultinomialNB() #complement naive bayes classifier with different smoothing parameters

    Y_pred = clf.fit(X_train, Y_train).predict(X_test) #fitting and predicting
    f1 = sk.metrics.f1_score(Y_test, Y_pred)
    print("f1 score is: ", f1)
    
    evaluate_model(Y_test,Y_pred)
    return clf


def predict_from_proba(pred_proba, threshold):
    lst = []
    
    for prob in pred_proba:
        if prob[1] >= threshold:
            lst.append(1)
        else:
            lst.append(0)        
    return np.array(lst)

def mismatched_indices(Y_test, Y_pred):
    ind_list = []
    for ind in range(len(Y_test)):
        if Y_test[ind] != Y_pred[ind]:
            ind_list.append(ind)
    return ind_list

In [53]:
all_lyrics_data = []

rap_file = ['rap'] #EDIT THESE LINES ACCORDING TO THE DIRECTORY AND FILE NAMES
nrap_files = ['nrap','nrap2','nrap3'] #EDIT THESE LINES ACCORDING TO THE DIRECTORY AND FILE NAMES
file_root = 'Dataset/AZlyrics/filtered/'#EDIT THESE LINES ACCORDING TO THE DIRECTORY AND FILE NAMES
file_format = '.csv' #EDIT THESE LINES ACCORDING TO THE DIRECTORY AND FILE NAMES

rap_label_list = []
nrap_label_list = []
rap_lyrics_list = []
nrap_lyrics_list = []
all_lyrics_list = []


#GET ALL LYRICS FROM RAP SONGS rap_lyrics_list, AND CREATE CORRESPONDING LABEL LIST rap_label_list
for ini in rap_file:
    file_name = file_root + ini + file_format 
    data = read_csv(file_name)
    
    all_lyrics_list = data['LYRICS'].values.tolist()
    rap_lyrics_list = data['LYRICS'].values.tolist()
    
    for entry in rap_lyrics_list:
        entry = entry.lower()
        entry = re.sub(r'[^a-zA-Z0-9\-\']',' ',entry)
        entry = re.sub(r'[\s,.?]+',' ',entry)
        rap_label_list.append(1)

#GET ALL LYRICS FROM NON-RAP SONGS nrap_lyrics_list, AND CREATE CORRESPONDING LABEL LIST nrap_label_list
for ini in nrap_files:
    sublist = []
    file_name = file_root + ini + file_format
    data = read_csv(file_name)
    
    sublist = data['LYRICS'].values.tolist()
    
    for entry in sublist:
        entry = entry.lower()
        entry = re.sub(r'[^a-zA-Z0-9\-\']',' ',entry)
        entry = re.sub(r'[\s,.?]+',' ',entry)
        nrap_label_list.append(0)
        
    all_lyrics_list = all_lyrics_list + sublist
    nrap_lyrics_list = nrap_lyrics_list + sublist
    
#ALSO SAVES ALL LYRICS (RAP AND NON-RAP) INTO all_lyrics_list

In [54]:
#MANUALLY FORCING THE TRAINING DATASET TO BE BALANCED
x=4 #ADJUSTABLE STARTING ELEMENT, X MUST BE SMALLER THAN 30
nrap_lyrics_list_sampled = nrap_lyrics_list[x::30]
all_lyrics_list_sampled = rap_lyrics_list + nrap_lyrics_list_sampled
label_list_sampled = rap_label_list + nrap_label_list[x::30]

In [55]:
#TF-IDF VECTORIZER, PARAMETERS ADJUSTABLE, FITTED TO ONLY RAP LYRICS, VECTORIZED FOR 
tfidf_dictionary = TfidfVectorizer(ngram_range=(1,1),max_df=0.6,max_features=5000,min_df=0.00002, token_pattern=r"[a-zA-Z0-9\-\']+").fit(all_lyrics_list)
lyrics_vectorized = tfidf_dictionary.transform(all_lyrics_list_sampled)

In [56]:
#SPLIT DATASET FOR TRAINING AND TESTING, PARAMETERS ADJUSTABLE
X_train, X_test, Y_train, Y_test = train_test_split(lyrics_vectorized, label_list_sampled,\
                                                   test_size=0.25, random_state=42)

In [57]:
mnb_train = mnb_simple(X_train, Y_train, X_test, Y_test)

f1 score is:  0.802775024777007
TP:  810
TN:  861
FP:  173
FN:  225
f1 score is:  0.802775024777007


In [60]:
survey = read_csv('Dataset/Generated/survey.csv')
survey_lyrics = survey['LYRICS'].values.tolist()

In [61]:

test_file_list = ['Dataset/Generated/mc_list.p',
                 'Dataset/Generated/onelyr_list.p',
                 'Dataset/Generated/random_list.p',
                 'Dataset/Generated/twolyr_list.p']

"""
test_file_list = ['mc_list.p',
                 'onelyr_list.p',
                 'random_list.p',
                 'twolyr_list.p']
"""
"""Change the above list accordingly, each file should contain a list of songs/documents (a list of lists)"""

collated_list = []

for f in range(len(test_file_list)):    
    pickle_file = open(test_file_list[f], "rb")

    while True:
        try:
            collated_list.append(pickle.load(pickle_file))
        except EOFError:
            break

"""below are the names of the lists of lyrics to be testing"""
mc = collated_list[0] 
onelyr = collated_list[1]
random = collated_list[2]
twolyr = collated_list[3]

In [62]:
#prepares the test set and labels
test_list = survey_lyrics #Change this line to one of the names of the test lists: mc/onelyr...
test_labels = []

for entry in test_list:
    test_labels.append(1)

test_vectorized = tfidf_dictionary.transform(test_list)
X_test = test_vectorized
Y_test = test_labels

In [65]:
mnb_clf = mnb_simple(X_train, Y_train, X_test, Y_test)
mnb_Y_pred_proba = mnb_clf.predict_proba(X_test)
#print(mnb_Y_pred_proba)

mnb_Y_pred = predict_from_proba(mnb_Y_pred_proba, 0.4)
#print(mnb_Y_pred)
evaluate_model(Y_test, mnb_Y_pred)

#prints out the lines which the classifier deems not rap
#wrong = mismatched_indices(Y_test, mnb_Y_pred)
#for ind in wrong:
    #print(ind)
    #print(test_list[ind])

f1 score is:  0.9500000000000001
TP:  38
TN:  0
FP:  0
FN:  4
f1 score is:  0.9500000000000001
TP:  41
TN:  0
FP:  0
FN:  1
f1 score is:  0.9879518072289156
